In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
prev_len = 0
links = set()
page = 1
while True:
    search = requests.get(f"https://www.mobelringen.no/stue/sofa?p={page}")
    soup = BeautifulSoup(search.content, 'html.parser')
    new_links = set(i.attrs['href'] for i in soup.find_all('a', attrs={'class':'product-item-link'}))
    prev_len = len(links)
    links |= new_links
    if prev_len == len(links):
        break
    page += 1
    

In [3]:
def scrap_mobelringen(url):
    #print (url)
    content = requests.get(url).content
    soup = BeautifulSoup(content, 'html.parser')
    price_amount = soup.find('meta', attrs={'property':'product:price:amount'}).attrs['content']
    price_currency = soup.find('meta', attrs={'property':'product:price:currency'}).attrs['content']
    model = soup.find('h1', attrs={'class':'page-title'}).text.strip()
    dims_el = soup.find('ul', attrs={'class':'dimensions desktop'})
    dims = dims_el.find_all('span', attrs={'class':'value'})

    item = {
        'source' : 'mobelringen',
        'model' : soup.find('meta', attrs={'property':'og:title'}).attrs['content'],
        'image' : soup.find('meta', attrs={'property':'og:image'}).attrs['content'],
        'description' : soup.find('meta', attrs={'property':'og:description'}).attrs['content'],
        'url' : soup.find('meta', attrs={'property':'og:url'}).attrs['content'],
        'price' : price_amount+' '+price_currency,
        'sku' : soup.find('meta', attrs={'itemprop':'sku'}).attrs['content'],
        'brand' : soup.find('meta', attrs={'itemprop':'brand'}).attrs['content']
    }

    trans = {'Bredde': 'width',
     'Høyde': 'height',
     'Lengde': 'length',
     'Sittehøyde': 'sit_height',
     'Dybde': 'depth'}

    dims_el = soup.find('ul', attrs={'class':'dimensions desktop'})
    dims_l = dims_el.find_all('li')
    for l in dims_l:
        label = l.find('label')
        if not label or not trans.get(label.text):
            continue
        val = l.find('span', attrs={'class':'value'}).text[:-3]
        item[trans.get(label.text)] = val
        try:
            item[trans.get(label.text)+"_i"] = int(val)
        except ValueError:
            pass
        
    item['sleep'] = 'n'
    return item

items = (scrap_mobelringen(url) for url in links)

In [177]:
df = pd.DataFrame(items)
df['sleep'] = df['model'].str.contains('sovesofa')
#df.to_csv("/home/veon/Documents/mobelringen.csv")

# Start here

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/veonua/norske_sofaer/master/data/mobelringen.csv")
len(df)

184

In [4]:
df['max_depth_i'] = df['length_i']

In [9]:
filt = df[['image', 'price', 'model', 'brand', 'max_depth_i', 'width_i', 'url']]
fw = df['width_i'].between(260, 285) # | (df['width_i'] == 0)
fd = df['max_depth_i'].between(200,223) | (df['max_depth_i'] == 0)
filt = filt[fd & fw].sort_values('brand')

In [14]:
from IPython.display import HTML
def image_formatter(im):
    return f'<img src="{im}">'
def a_formatter(url):
    return f'<a href="{url}">link</a>'

HTML(filt.to_html(formatters={'image': image_formatter, 'url': a_formatter}, escape=False))

,image,price,brand,max_depth_i,width_i,url
102,,9999 NOK,DOMINARI,202.0,280.0,link
116,,9999 NOK,DOMINARI,202.0,280.0,link
113,,14719 NOK,DOMINARI - NORTH LIVING,206.0,279.0,link
121,,14719 NOK,DOMINARI - NORTH LIVING,206.0,279.0,link
4,,28699 NOK,HJELLEGJERDE AS,202.0,280.0,link
69,,30499 NOK,HJELLEGJERDE AS,202.0,270.0,link
160,,28699 NOK,HJELLEGJERDE AS,202.0,280.0,link
176,,29299 NOK,HJELLEGJERDE AS,218.0,275.0,link
28,,14999 NOK,SCAPA BALTIC UAB,210.0,260.0,link
172,,14999 NOK,SCAPA BALTIC UAB,210.0,260.0,link
